<a href="https://colab.research.google.com/github/Usermer/deep-learning-universe/blob/main/02_deep_learning_with_keras_and_tensorfow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#perceptron
import numpy as np
import pandas as pd
from sklearn.linear_model import Perceptron
from sklearn.datasets import load_iris

RANDOM_STATE=42

In [2]:
iris=load_iris(as_frame=True)
iris.data.head()
iris_df=iris.data.copy()
iris_df["target"]=iris.target
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [3]:
X=iris.data[["petal length (cm)","petal width (cm)"]].values
y=(iris.target==0)#Iris setosa

perceptron_clf=Perceptron(random_state=RANDOM_STATE)
perceptron_clf.fit(X,y)


X_new=[[1.5,0.6]]
y_pred=perceptron_clf.predict(X_new)
y_pred

array([ True])

**Regression MLPs**

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

housing=fetch_california_housing()
X=housing.data
y=housing.target
X_train_full,X_test,y_train_full,y_test=train_test_split(X,y,test_size=0.2,random_state=RANDOM_STATE)
X_train,X_val,y_train,y_val=train_test_split(X_train_full,y_train_full,test_size=0.2,random_state=RANDOM_STATE)

mlp_reg=MLPRegressor(hidden_layer_sizes=[50,50,50],random_state=RANDOM_STATE)
pipeline=make_pipeline(StandardScaler(),mlp_reg)
pipeline.fit(X_train,y_train)
y_pred=pipeline.predict(X_val)
rmse=np.sqrt(mean_squared_error(y_val,y_pred))
rmse

**Implementing MLPs with Keras**

***building an Image Classifiert Using the Sequential API***

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets
from tensorflow.keras import Sequential

RANDOM_STATE=42
tf.keras.utils.set_random_seed(RANDOM_STATE)

Training a neural network involves randomness in many places:

*  Weight initialization



*  Shuffling training data

*  Dropout

*  Some GPU operations

In [ ]:
fashion_mnist=datasets.fashion_mnist.load_data()


In [ ]:
(X_train_full,y_train_full),(X_test,y_test)=fashion_mnist
X_train,y_train=X_train_full[:50000],y_train_full[:50000]
X_val,y_val=X_train_full[50000:],y_train_full[50000:]


In [ ]:
X_train.shape


When loading MNIST on Fashion MNIST using Keras rather than Scikit-learn , one important difference is that every image is represented as a 28 x 28 array rather than 1D array of size:784

In [ ]:
#X_train[1]
y_train[1]

For simplicity, we'll scale the pixel intensities from 0-1 range rather that 0-255 by dividing thel by 255

In [ ]:
X_train,X_val,X_test=X_train/255,X_val/255,X_test/255

For fashion-mnist we need a list of class names to know what we are dealing with:

In [ ]:
class_names=["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]

**Creatig the model using the `sequential` API**

**Sequential API** :is the simplest way to build neural networks when layers are stacked one after another.

In [ ]:
model=Sequential([
    keras.layers.Input(shape=[28,28],name="input"),
    keras.layers.Flatten(),
    keras.layers.Dense(300,activation='relu',name="dense_1"),
    keras.layers.Dense(100,activation="relu",name="dense_2"),
    keras.layers.Dense(10,activation="softmax",name="output")
])



In [ ]:
#the layers are accessible via layers attribute
model.layers

**A Sequential model behaves very much like an array of layers**

In [ ]:
#model.pop()
#model.add(keras.layers.Dense(10,activation="softmax"))
len(model.layers) #flatten+3 dense layers

In [ ]:
model.summary()

In [ ]:
dense_1=model.layers[1]
weights,bias=dense_1.get_weights()
weights #random in the start

🔹 Why are the weights random at the start?

**To break symmetry:**

If all weights started the same:

*  All neurons would learn the same thing

*  The network would be useless

**Random initialization ensures:**
Each neuron learns different features

In [ ]:
bias#0

**Compiling the model**

In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="sgd",
    metrics=["accuracy"]
)

**Training and evaluationg the model**

In [ ]:
history=model.fit(
    X_train,y_train,
    epochs=30,
    validation_data=(X_val,y_val)
)